Welcome to my project !
in this following notebook you would encounter some practices about:
reciving JSON response from REST API of a mobile game - Disney Sorcerers Arena.
translating the response to normalized tables with pandas.


That an RPG Game which focus on wide roster of characters and offer PVP and PVE modes.
The purpose of this project is to create rdb for analysis and bots creation.

I wish to empower my skills in engineering.
in this notebook you will find practice about how to extract crucial data about the character power.

In [1]:
import numpy as np,pandas as pd,json,sqlite3,requests
from jsonpath_ng import jsonpath
from jsonpath_ng.ext import parse
from pandas.io.json import json_normalize
pd.options.display.max_columns = 99

with open('/Users/mac/Documents/Coding/Projects/DSA/Resources/characters_in_dsa.json', 'r') as char_data:
    chars_json = json.load(char_data)



After loading the source file as python object, I found this meta deta:
- there are 176 main arrays. each array holds all the data over a character.
let's remind that each array evaluate to more arrays of key-value, and value can become another array (hello nested).

I'd use serieses and DataFrames constantly to avoid memory and time consuming of slower classes in python.

In [4]:
print(type(chars_json)) #dict
print(len(chars_json)) #176
toons_ids = pd.Series(chars_json.keys())
print(type(toons_ids))
print(toons_ids[0])

<class 'dict'>
176
<class 'pandas.core.series.Series'>
7266520144288097


First, We need to figure out what we can extract, and looking at the first level We only got the toons ID's. let's dive deeper. I want to keep my script as reusable as possible so I try to avoid the singular data usage. I took all the id's and created a list of the values, so I can take the first position and work with that.

In [5]:
demo = chars_json[toons_ids[0]]
print(demo.keys())

dict_keys(['ID', 'NameKey', 'UnitStats', 'StatProgressionTable', 'GearSlotTable', 'GearTiers', 'Icon', 'Obtainable', 'Abilities', 'PromotionMaterial', 'UnlockRarity', 'WalkSpeed', 'Categories', 'GearTierMeritItems', 'Skins', 'Hidden'])


That some more helpful keys. I want to construct a table of the toons. Thanks to my experience in the game my data will be:
ID | NameKey | StatProgressionTable | Obtainable | UnlockRarity | *Categories

The categories level is another dict that needs to be flattened. I wish to make it:
ID | NameKey | StatProgressionTable | Obtainable | UnlockRarity | Class | Fiction | Story.

Let's see what's going on there.

In [8]:
rel_cols = ['ID', 'NameKey','Obtainable', 'UnlockRarity','Categories','StatProgressionTable']

toon_df = pd.json_normalize(demo)
categories_series = demo['Categories']
pd.json_normalize(chars_json[toons_ids[0]],record_path=['Categories'],meta=['ID', 'NameKey','Obtainable', 'UnlockRarity','StatProgressionTable'])

,0,ID,NameKey,Obtainable,UnlockRarity,StatProgressionTable
0,614606076824576,7266520144288097,SHERIFF_OF_NOTTINGHAM_UNIT_NAME,True,3,6619604581022230
1,5960529125021754,7266520144288097,SHERIFF_OF_NOTTINGHAM_UNIT_NAME,True,3,6619604581022230
2,4419393874676815,7266520144288097,SHERIFF_OF_NOTTINGHAM_UNIT_NAME,True,3,6619604581022230
3,4934108704849804,7266520144288097,SHERIFF_OF_NOTTINGHAM_UNIT_NAME,True,3,6619604581022230
4,2136312807087658,7266520144288097,SHERIFF_OF_NOTTINGHAM_UNIT_NAME,True,3,6619604581022230


So our demo toon is The sherrif, so far we have nice dataframe which holds the categories as values. our next phase will be to check each cat_id and to determine what is it and to switch the id for the name.

There is another JSON file that holds all the terms and their translation.

In [ ]:
with open('/Users/mac/Documents/Coding/Projects/DSA/Resources/game_data_in_dsa.json', 'r') as game_data_file: